## LLM Router Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
llm = ChatOpenAI()

## Establish the templates for the two routes

In [ ]:
beginner_template = "You are a linguistics 101 instructor. You want to explain complex liguistics concepts in the simplest ways possible. You assume no prior knowledge of linguistics. Here is your question: \n{input}"
advanced_template = "You are a linguistics professor who is on a panel at an academic conference. You can assume that anyone who asks you a question has, or is getting, a graduate degree in linguistics. Here is your question: \n{input}"

## Establish route prompt information

In [ ]:
prompt_infos = [
    {"name":"beginner linguistics", 
    "description": "answers basic linguistics questions",
    "template":beginner_template,},
        {"name":"advanced linguistics", 
    "description": "answers advanced linguistics questions",
    "template":advanced_template,},
]
# the data type is list

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm =llm, prompt=prompt)
    destination_chains[name] = chain

In [ ]:
type(destination_chains)
# type should be dict (curly braces)


## Establish default prompt

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
# {input} refers back to the two route templates
default_chain = LLMChain(llm=llm, prompt=default_prompt)

## Set up routing

In [ ]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)
# this is an elaborate prompt that instructs the LLM how to do the routing. It needs to be modified.

In [ ]:
destinations = [f"{p['name']}: {p['description']}"  for p in prompt_infos]

In [ ]:
destination_str = "\n".join(destinations)
# This is how it has to be formatted to fit into the multi prompt router template: not as two strings, but as one string with a line break in between

In [ ]:
print(destination_str)

## Creating router template

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destination_str
)

In [ ]:
print(router_template)
# the router template now has the appropriate routing information

## Creating router prompt

In [ ]:
router_prompt = PromptTemplate(template= router_template,
                               input_variables = ['input'],
                               output_parser = RouterOutputParser())

## Creating router chain

In [ ]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains = destination_chains,
                         default_chain=default_chain,
                         verbose = True)

## Running the chain

In [ ]:
print(chain.run("what's a phoneme?"))

In [ ]:
print(chain.run("What are the cognitive advantages to right-branching languages?"))
# it's not activating the advanced chain for some reason